# Formats acceptés par Xsuite

Ce notebook donne une vue des différentes façons de créer, importer et exporter des mailles avec Xsuite.

In [ ]:
import xtrack as xt
from cpymad.madx import Madx

%matplotlib widget

## Charger une maille MAD-X via cpymad

Le paquet [cpymad](https://pypi.org/project/cpymad/) permet de communiquer avec le process MAD-X et de programmer les simulations via Python. Cette approche permet de gérer tout script MAD-X, y compris l'inclusion et l'exécution de macros. Cette approche sera discontinuée à la fin du support de MAD-X (~2029).

In [ ]:
%%capture
madx = Madx()
madx.call("inputs/pimms.madx")
madx.beam()
madx.use("pimms")

In [ ]:
# La séquence cpymad elle-même peut être importée dans Xsuite
ring = xt.Line.from_madx_sequence(madx.sequence.pimms, deferred_expressions=True)
ring.particle_ref = xt.Particles(kinetic_energy0=200e6)

In [ ]:
twiss = ring.twiss4d()  # Twiss
twiss.rows["qfa.*"].cols["s betx bety dx"]  # optique à une famille de quadripôles focalisants

## Charger séquence et optique directement depuis des scripts MAD-X

Xsuite inclus un analyseur syntaxique qui peut directement ingérer des scripts MAD-X définissant des séquences, références etc. Cette approche ne gère pas l'exécution de macros. L'approche sera discontinuée à la fin du support MAD-X (~2029).

In [ ]:
env = xt.load("inputs/pimms.madx")
ring = env["pimms"]
ring.particle_ref = xt.Particles(kinetic_energy0=200e6)

In [ ]:
twiss = ring.twiss4d()
twiss.rows["qfa.*"].cols["s betx bety dx"]  # optique à une famille de quadripôles focalisants

### Charger une maille en JSON

In [ ]:
env = xt.Environment.from_json("inputs/pimms.json")
twiss = env.pimms.twiss4d()  # Twiss
twiss.rows["qfa.*"].cols["s betx bety dx"]  # Get optics at selected locations

### Charger un environement défini de manière programmatique (Python)

Il est possible de définir la maille ainsi que les références et alimentations directement en script Python (ce qui peut inclure logique, commentaires, etc.) et de les charger directement dans Xsuite.

Cette solution fonctionne avec les mailles très larges, comme par exemple la [maille du LHC](https://raw.githubusercontent.com/xsuite/xtrack/refs/heads/main/examples/lhc_native_python/lhc_seq.py).

In [ ]:
%%capture
env = xt.Environment()
env.call("inputs/pimms.py")

In [ ]:
twiss = env.ring.twiss4d()  # Twiss
twiss.rows["qfa.*"].cols["s betx bety dx"]  # Get optics at selected locations

## Charger des configurations hébergées

Il est possible de charger des configurations hébergées en ligne, pourvu qu'elles respectent les formats ci-dessus. Ici par exemple, nous pouvons très facilement charger la configuration complète du LHC en fin de beta-star levelling, pendant les collisions.

In [ ]:
lhc = xt.load("https://acc-models.web.cern.ch/acc-models/lhc/2025/xsuite/lhc.json")
lhc.vars.load("https://acc-models.web.cern.ch/acc-models/lhc/2025/scenarios/cycle/pp/levelling/889/optics.madx")
lhc.b1.particle_ref = xt.Particles(mass0=938272046.0, q0=1, energy0=6800000000000.0)
lhc.b2.particle_ref = xt.Particles(mass0=938272046.0, q0=1, energy0=6800000000000.0)

---